In [1]:
! pip install transformers


     |████████████████████████████████| 3.5 MB 11.7 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 42.9 MB/s 
     |████████████████████████████████| 895 kB 42.1 MB/s 
     |████████████████████████████████| 6.8 MB 42.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import string
from nltk.tokenize import WordPunctTokenizer
from string import digits, ascii_lowercase, punctuation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModel
import torch
from gensim.models import KeyedVectors
import pickle
import nltk

from collections import Counter

import pandas as pd

from tqdm import tqdm


# Данные и препроцессинг

In [3]:
! wget https://raw.githubusercontent.com/jeka-e/WASSA2022_EMO/main/messages_dev.tsv
! wget https://raw.githubusercontent.com/jeka-e/WASSA2022_EMO/main/messages_train.tsv
! wget https://raw.githubusercontent.com/jeka-e/WASSA2022_EMO/main/goldstandard_dev_2022.tsv

--2022-02-12 19:25:20--  https://raw.githubusercontent.com/jeka-e/WASSA2022_EMO/main/messages_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131326 (128K) [text/plain]
Saving to: ‘messages_dev.tsv’

messages_dev.tsv    100%[===================>] 128.25K  --.-KB/s    in 0.009s  

2022-02-12 19:25:21 (14.3 MB/s) - ‘messages_dev.tsv’ saved [131326/131326]

--2022-02-12 19:25:21--  https://raw.githubusercontent.com/jeka-e/WASSA2022_EMO/main/messages_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1100753 (1.0M) [tex

In [4]:
all_data_train = pd.read_csv('messages_train.tsv', sep="\t")
goldstandard = pd.read_csv('goldstandard_dev_2022.tsv', sep="\t", names=[str(i) for i in range(12)])

all_data_val = pd.read_csv('messages_dev.tsv', sep="\t")
x_val = all_data_val[['essay']]
y_val = goldstandard[['2']]  # only emotion label column
y_val.columns = ['emotion']
# x_val['emotion'] = y_val

x_train = all_data_train[['essay']] # leave columns related to this track
y_train = all_data_train[['emotion']] # only emotion label column

In [5]:
em2id = {'neutral': 0, 
         'sadness': 1,
         'anger': 2, 
         'fear': 3, 
         'surprise': 4, 
         'disgust': 5, 
         'joy': 6
}

id2em = {i:word for word, i in em2id.items()}

y_val = np.array(y_val['emotion'].map(em2id))
y_train = np.array(y_train['emotion'].map(em2id))


# Векторизация

## FastText

In [6]:
ft_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/fasttext_en/wiki-news-300d-1M.vec')
# загружается сто лет, около 5 минут... (даже на гпу)


In [7]:
def clean(text): 

    #lowercase the text
    text = text.lower()
    
    #remove punctuations
    text = "".join([w for w in text if not w in string.punctuation])
    
    #split the text
    # text = text.split()
    
    #remove stopwords
    # text = " ".join([word for word in text if not word in stop_words])
    
    return text

In [8]:
def get_emb_ft(sent, model):
    sent_vector = np.zeros(300)
    for word in sent.split():
        try:
            sent_vector += ft_model.wv[word]
        except KeyError:
            # oov zero vector
            continue
    return sent_vector/len(sent.split())

In [9]:
x_train_ft = x_train['essay'].apply(clean)
x_val_ft = x_val['essay'].apply(clean)

In [10]:
weights_ft = np.zeros((len(x_train_ft), 300))

for i, sent in enumerate(tqdm(x_train['essay'])):
    sent_vector = get_emb_ft(sent, ft_model)
    weights_ft[i] = sent_vector

  0%|          | 0/1860 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
100%|██████████| 1860/1860 [00:01<00:00, 1074.64it/s]


In [11]:
weights_val_ft = np.zeros((len(x_val_ft), 300))

for i, sent in enumerate(tqdm(x_val['essay'])):
    sent_vector = get_emb_ft(sent, ft_model)
    weights_val_ft[i] = sent_vector

  0%|          | 0/270 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
100%|██████████| 270/270 [00:00<00:00, 1321.16it/s]


In [12]:
# with open('weights_val_ft.pickle', 'wb') as f:
#     pickle.dump(weights_val_ft, f)


## BERT

In [13]:
bert_model = AutoModel.from_pretrained('bhadresh-savani/bert-base-uncased-emotion')
tokenizer = AutoTokenizer.from_pretrained('bhadresh-savani/bert-base-uncased-emotion')

Downloading:   0%|          | 0.00/935 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at bhadresh-savani/bert-base-uncased-emotion were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/285 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
def get_emb_bert(sent, model, tokenizer):
    with torch.no_grad():
      enc = tokenizer(sent, return_tensors='pt')
      output = model(**enc, return_dict=True)
    # vector = output.last_hidden_state[:,0,:].numpy()   #CLS token
    vector = torch.mean(output.last_hidden_state, dim=1)
    return np.squeeze(vector)

In [15]:
weights_train_bert = np.zeros((len(x_train), 768))

for i, sent in enumerate(tqdm(x_train['essay'])):
    sent_vector = get_emb_bert(sent, bert_model, tokenizer)
    weights_train_bert[i] = sent_vector

100%|██████████| 1860/1860 [13:38<00:00,  2.27it/s]


In [16]:
weights_val_bert = np.zeros((len(x_val), 768))

for i, sent in enumerate(tqdm(x_val['essay'])):
    sent_vector = get_emb_bert(sent, bert_model, tokenizer)
    weights_val_bert[i] = sent_vector

100%|██████████| 270/270 [01:56<00:00,  2.31it/s]


In [17]:
# with open('weights_val_bert.pickle', 'wb') as f:
#     pickle.dump(weights_val_bert, f)
